In [1]:
# Initialization. Auto update changed modules so the kernel does not need to be restarted after changes
%load_ext autoreload
%autoreload 2

# Set device udid. If your device udid is different, run adb devices and change the udid
device_udid = "emulator-5554"


# Exercise 2: Add new functionality
In this exercise, you will add new functionality to the camera app.
Currently, the camera class can only take a picture. In this exercise, we will learn how to switch the camera
and take a video.

Try to execute below code, what happens?


In [6]:
import os, time
os.chdir('../..')
from puma.apps.android.open_camera.open_camera import OpenCameraActions

camera = OpenCameraActions(device_udid)

camera.take_picture()
time.sleep(2)
camera.switch_camera()
time.sleep(2)
camera.take_picture()

`switch_camera` has not been implemented yet. That is what you will do in this exercise. When adding new
functionality to Puma, you will follow the following steps:
- Manually exsplore the actions you want to automate on the device to get an idea of the steps you need to
  implement
- Use Appium Inspector to identify the UI elements you want to interact with
- Write the Appium code to select the element and interact with it using XPATH

We will take you through these steps below.


# How Appium interacts with the App UI

Before we get started we need to udnerstand how Appium interacts with the UI. On Android the UI consists
of elements in a hierarchy, much like an XML document. Appium fetches this hierarchy through the Android
Accesability API and exposes this as an XML document to the user. Usually, only visible elements are
included, as invisible elements should not be loaded into the Android Accesibility service. However, not
all apps are equally well written, and some apps laod in elements too early, while other apps do not
load relevant data into the Accesability Service, leaving us navigating XML elements without any contents
or attributes.

Lucky for us, most of the time there's a few attributes we can use to interact with the desired UI element.
Let's take a look at the following UI:

<img src="resources/whatsapp.png" alt="Alt Text" width="300">

If we retrieve the XML element representing the send button (how we do that will be explained later in this
exercise), it loks like this:
```XML
<android.widget.FrameLayout resource-id="com.whatsapp:id/send_container">
    <android.widget.ImageButton content-desc="Send" resource-id="com.whatsapp:id/send"/>
<android.widget.FrameLayout/>
```
As you can see, we've the button itself is an `ImageButton` element nested inside a `FrameLayout` element.
The good news is that either can be clicked to interact with the button, so we don't need to worry about that.
You could idenitify the elements based on the element class, but in this case this will only work when there's
just one `ImageButton`. As UIs can become very complex, we usually want something more spcific to go by, which
is why we usually also look at attributes.
Either of these elements can be identified by their `resource-id`, and the inner element also has a
`content-desc` which we can use to identify it. Of these two, the preference usually goes to `resource-id`, as
this value is indendant of the configured system language.

If we take a look at one of the sent messages, we see the following XML, containing a third attribute that can
be useful:
```XML
<android.widget.FrameLayout resource-id="com.whatsapp:id/conversation_text_row">
    <android.widget.TextView text="Hello there Alice, long time no see! How have you been??" resource-id="com.whatsapp:id/message_text"/>
        <android.widget.LinearLayout resource-id="com.whatsapp:id/date_wrapper">
            <android.widget.TextView text="1:21 PM" resource-id="com.whatsapp:id/date"/>
            <android.widget.ImageView content-desc="Delivered" resource-id="com.whatsapp:id/status"/>
        </android.widget.LinearLayout>
    </android.widget.FrameLayout>
</android.widget.LinearLayout>
```
Here we can read the message contents in the `text` attribute, as well as the time the message was sent. Keep 
in mind that the `text` attribute is even more volatile than the `content-desc`, as it might be determined by
both the system language and user input.

So, to summarize:
* The Android UI is exposed as an XML document through Appium
* Selecting elements can be done by looking at the element class and the attribute values
* In practice, the attributes `resource-id`, `content-desc` and `text` prove to be the most useful

# Appium Python Client
To use Appium in Python, we use the Appium Python Client. The Appium Python Client works by sending commands to the Appium Server, which then communicates with the emulator. The interaction follows the WebDriver protocol, allowing automation of mobile applications similarly to how Selenium automates web browsers. First a webdriver session is initialized (see the AndroidAppiumActions class for the implementation). The webdriver has some functions to interact with elements, as well as retrieving information:
```python
# Open an app
driver.activate_app(app_package)

# Press the Android home key
driver.press_keycode(AndroidKey.HOME)

# Find an element in the current view
driver.find_element(by=AppiumBy.XPATH, value=xpath)

# Swipe from one coordinate to another
driver.swipe(start_x, start_y, end_x, end_y)
```

Commands can be executed on the elements, such as clicking and sending keys (_i.e._ typing text in a text box.):
```python
shutter_element = self.driver.find_element(by=AppiumBy.XPATH, value=xpath)
shutter_element.click()

text_box = self.driver.find_element(by=AppiumBy.XPATH, value=xpath)
text_box.send_keys("This is the text to insert into the text box")
```


# Finding XML elements
As seen in the previous example, finding elements can be done with `webdriver`.find_element(). The element can be found in different ways:

```python
driver.find_element(by=AppiumBy.ID, value="id")
driver.find_element(by=AppiumBy.CLASS_NAME, value="class_name")
driver.find_element(by=AppiumBy.ACCESSIBILITY_ID, value="accessibility_id")
driver.find_element(by=AppiumBy.XPATH, value="xpath")
```

As you can see, there are several ways to find elements. The first 3 are based on selecting a specific attribute. The last one in based on XPath. In Puma, we prefer to use XPath, as it enables us to make more complex queries to select elements. Moreover, when something changes in a new version of the application, only the value needs to be updated.


# Appium inspector
Appium Inspector is a GUI assistant tool for Appium, providing visual inspection of the application.
1. Open Appium inspector.
2. Under `Saved Capability Sets`, select `Emulator` and press `Start Session`. If you do not see saved capabilities, please refer to `workshop/resources/appium_inspector_capability_set.json` and add it.
You now see your emulator screen on the left pane, here you can select elements. In the middle pane you see the xml representation of the current view in the app. On the right pane you see the information about the selected element.
3. On the emulator, open the camera app and make sure you are in the home screen. Refresh AppiumInspector to update the screen if necessary.
4. In Appium inspector, select the #TODO element in the top-right corner and look at the information about the element in the right pane.







<details>
  <summary style="font-size: 24px;">What is the value of the class attribute of this element?TODO answer</summary>
  TODO
</details>

<details>
  <summary style="font-size: 24px;">What attribute has the most uniquely identifying value?TODO answer</summary>
  TODO
</details>


TODO Move this info on the right location
Choose a unique attribute to select the element on. `content-desc` is usually the preferred attribute, but is not always
present, or does not always have a uniquely identifying value. If that is the case, choose another attribute with a uniquely identifying value, such as the resource id or text. Appium Inspector gives a suggestion
for the XPath to use. If the attribute value is language dependent, (e.g., "send message" vs. "stuur bericht"),
ensure that it supports English.


# XPath expressions
XML Path Language (XPath) provides a way to navigate through XML elements and attributes. TODO further explain

Implement switch view to make a selfie

Advanced: zoom in/out by pinching